In [27]:

"""
Assignment 2, Task 2: Object Detection (Full Comparison)
- Compares Faster R-CNN (two-stage) and YOLOv8n (single-stage).
- Measures FPS on a video file.
- Measures inference time on a single image.
- Saves visualization outputs.
- Prints a final comparison table.
"""



'\nAssignment 2, Task 2: Object Detection (Full Comparison)\n- Compares Faster R-CNN (two-stage) and YOLOv8n (single-stage).\n- Measures FPS on a video file.\n- Measures inference time on a single image.\n- Saves visualization outputs.\n- Prints a final comparison table.\n'

In [28]:
import torch
import torchvision.models.detection as detection
import torchvision.transforms as T
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import cv2
import time
import os
import numpy as np
import pandas as pd

# --- 0. Setup ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# !!! --- SET YOUR FILE PATHS HERE --- !!!
TEST_IMAGE_PATH = "coco128/images/train2017"  # A .jpeg image for detection
TEST_VIDEO_PATH = "test.gif"  # A short .gif video for FPS testing
# !!! --------------------------------- !!!


output_dir = "task_2_outputs"
os.makedirs(output_dir, exist_ok=True)



Using device: cuda


In [29]:
# COCO class names (for Faster R-CNN)
COCO_CLASSES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# --- 1. Model Loading ---

def load_faster_rcnn_model():
    """Loads a pre-trained Faster R-CNN model in evaluation mode."""
    print("Loading Faster R-CNN (ResNet-50 FPN)...")
    model = detection.fasterrcnn_resnet50_fpn(weights='COCO_V1')
    model.to(DEVICE).eval()
    print("Model loaded.")
    return model

def load_yolo_model(model_name='yolov8n.pt'):
    """Loads a pre-trained YOLO model."""
    print(f"Loading YOLO ({model_name})...")
    model = YOLO(model_name)
    model.to(DEVICE)
    print("Model loaded.")
    return model

# --- 2. Helper Functions ---

def get_model_size(model_path):
    """Gets the model size in MB."""
    if not os.path.exists(model_path):
        return "N/A (file not found)"
    return f"{os.path.getsize(model_path) / (1024 * 1024):.2f} MB"

def draw_boxes(image, boxes, labels, scores, threshold=0.5):
    """Draws bounding boxes on a PIL image."""
    img_draw = ImageDraw.Draw(image)
    for box, label, score in zip(boxes, labels, scores):
        if score > threshold:
            img_draw.rectangle(list(box), outline="red", width=3)
            img_draw.text((box[0], box[1]), f"{label} {score:.2f}", fill="red")
    return image



In [30]:
# --- 3. Inference and Benchmarking ---

def run_detection_frcnn(model, image_path):
    """Runs Faster R-CNN on a single image and returns annotated image + time."""
    img_pil = Image.open(image_path).convert("RGB")
    transform = T.Compose([T.ToTensor()])
    img_tensor = transform(img_pil).to(DEVICE)
    
    start_time = time.time()
    with torch.no_grad():
        prediction = model([img_tensor])
    end_time = time.time()
    
    inference_time = (end_time - start_time) * 1000  # in ms
    
    boxes = prediction[0]['boxes'].cpu().numpy()
    labels = [COCO_CLASSES[i] for i in prediction[0]['labels'].cpu().numpy()]
    scores = prediction[0]['scores'].cpu().numpy()
    
    annotated_img = draw_boxes(img_pil, boxes, labels, scores)
    return annotated_img, inference_time

def run_detection_yolo(model, image_path):
    """Runs YOLO on a single image and returns annotated image + time."""
    start_time = time.time()
    results = model(image_path, verbose=False)
    end_time = time.time()
    
    inference_time = (end_time - start_time) * 1000  # in ms
    
    # YOLO's plot() method is the easiest way to visualize
    annotated_img_cv = results[0].plot()  # Returns a NumPy array (BGR)
    annotated_img_pil = Image.fromarray(cv2.cvtColor(annotated_img_cv, cv2.COLOR_BGR2RGB))
    return annotated_img_pil, inference_time

def measure_fps(model, video_path, max_frames=100):
    """Measures average FPS on a video file."""
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Error: Could not open video {video_path}")
        return 0.0

    is_yolo = isinstance(model, YOLO)
    frame_count = 0
    total_time = 0
    
    print(f"Benchmarking FPS on {max_frames} frames...")
    
    while frame_count < max_frames:
        ret, frame = video.read()
        if not ret:
            break
        
        start_time = time.time()
        
        if is_yolo:
            _ = model(frame, verbose=False) # YOLO can take CV2 frames
        else:
            # Faster R-CNN needs PIL/Tensor
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            transform = T.Compose([T.ToTensor()])
            img_tensor = transform(img_pil).to(DEVICE)
            with torch.no_grad():
                _ = model([img_tensor])
                
        end_time = time.time()
        
        # Skip first frame for warmup
        if frame_count > 0:
            total_time += (end_time - start_time)
            
        frame_count += 1
        
    video.release()
    avg_fps = (frame_count - 1) / total_time if total_time > 0 else 0
    return avg_fps



In [ ]:
# --- 4. Main Execution Block ---

if __name__ == "__main__":
    
    if not os.path.exists(TEST_IMAGE_PATH) or not os.path.exists(TEST_VIDEO_PATH):
        print(f"Error: Please set valid paths for TEST_IMAGE_PATH and TEST_VIDEO_PATH.")
    else:
        results = []

        # --- Faster R-CNN ---
        print("\n" + "-"*30 + "\nTesting Faster R-CNN\n" + "-"*30)
        frcnn_model = load_faster_rcnn_model()
        # If TEST_IMAGE_PATH is a directory, run on up to 10 images inside; otherwise run single image
        if os.path.isdir(TEST_IMAGE_PATH):
            img_files = [f for f in sorted(os.listdir(TEST_IMAGE_PATH))
                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            img_files = img_files[:10]
            times = []
            last_ann = None
            for i, fname in enumerate(img_files):
                img_path = os.path.join(TEST_IMAGE_PATH, fname)
                ann_img, t = run_detection_frcnn(frcnn_model, img_path)
                out_name = f"frcnn_output_{i+1}.jpg"
                ann_img.save(os.path.join(output_dir, out_name))
                times.append(t)
                last_ann = ann_img
                print(f"Processed {fname}: {t:.2f} ms -> saved {out_name}")
            # Use last annotation for single-file expected later and average time
            frcnn_img = last_ann if last_ann is not None else Image.new("RGB", (1,1))
            frcnn_time = (sum(times) / len(times)) if times else 0.0
        else:
            frcnn_img, frcnn_time = run_detection_frcnn(frcnn_model, TEST_IMAGE_PATH)
        frcnn_img.save(os.path.join(output_dir, "frcnn_output.jpg"))
        print(f"Single Image Inference: {frcnn_time:.2f} ms")
        
        frcnn_fps = measure_fps(frcnn_model, TEST_VIDEO_PATH)
        print(f"Average Video FPS: {frcnn_fps:.2f}")
        
        results.append({
            "Model": "Faster R-CNN",
            "Type": "Two-Stage",
            "Size": "163 MB (COCO weights)",
            "FPS (Video)": f"{frcnn_fps:.2f}",
            "Inference (Image)": f"{frcnn_time:.2f} ms",
            "Output": "frcnn_output.jpg"
        })
        del frcnn_model # Free up GPU memory
        torch.cuda.empty_cache()

        # --- YOLO ---
        
        print("\n" + "-"*30 + "\nTesting YOLOv8n\n" + "-"*30)
        yolo_model_name = 'yolov8n.pt'
        yolo_model = load_yolo_model(yolo_model_name)
        if os.path.isdir(TEST_IMAGE_PATH):
            img_files = [f for f in sorted(os.listdir(TEST_IMAGE_PATH))
                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            img_files = img_files[:10]
            times = []
            last_ann = None
            for i, fname in enumerate(img_files):
                img_path = os.path.join(TEST_IMAGE_PATH, fname)
                ann_img, t = run_detection_yolo(yolo_model, img_path)
                out_name = f"yolo_output_{i+1}.jpg"
                ann_img.save(os.path.join(output_dir, out_name))
                times.append(t)
                last_ann = ann_img
                print(f"Processed {fname}: {t:.2f} ms -> saved {out_name}")
            # Use last annotation for single-file expected later and average time
            yolo_img = last_ann if last_ann is not None else Image.new("RGB", (1,1))
            yolo_time = (sum(times) / len(times)) if times else 0.0
        else:
            yolo_img, yolo_time = run_detection_yolo(yolo_model, TEST_IMAGE_PATH)
        yolo_img.save(os.path.join(output_dir, "yolo_output.jpg"))
        print(f"Single Image Inference: {yolo_time:.2f} ms")

        yolo_fps = measure_fps(yolo_model, TEST_VIDEO_PATH)
        print(f"Average Video FPS: {yolo_fps:.2f}")

        results.append({
            "Model": "YOLOv8n",
            "Type": "Single-Stage",
            "Size": get_model_size(yolo_model_name),
            "FPS (Video)": f"{yolo_fps:.2f}",
            "Inference (Image)": f"{yolo_time:.2f} ms",
            "Output": "yolo_output.jpg"
        })

        # --- 5. Final Report ---
        print("\n\n" + "="*50)
        print("      ASSIGNMENT 2 - FINAL COMPARISON REPORT")
        print("="*50)
        
        report_df = pd.DataFrame(results)
        print(report_df.to_string(index=False))
        
        print("\n" + "="*50)
        print(f"All detection outputs saved to '{output_dir}' directory.")


        


------------------------------
Testing Faster R-CNN
------------------------------
Loading Faster R-CNN (ResNet-50 FPN)...
Model loaded.
Processed 000000000009.jpg: 507.29 ms -> saved frcnn_output_1.jpg
Processed 000000000025.jpg: 165.10 ms -> saved frcnn_output_2.jpg
Processed 000000000030.jpg: 158.96 ms -> saved frcnn_output_3.jpg
Processed 000000000034.jpg: 153.89 ms -> saved frcnn_output_4.jpg
Processed 000000000036.jpg: 142.94 ms -> saved frcnn_output_5.jpg
Processed 000000000042.jpg: 147.28 ms -> saved frcnn_output_6.jpg
Processed 000000000049.jpg: 143.89 ms -> saved frcnn_output_7.jpg
Processed 000000000061.jpg: 141.27 ms -> saved frcnn_output_8.jpg
Processed 000000000064.jpg: 144.13 ms -> saved frcnn_output_9.jpg
Processed 000000000071.jpg: 152.78 ms -> saved frcnn_output_10.jpg
Single Image Inference: 185.75 ms
Benchmarking FPS on 100 frames...
Average Video FPS: 6.18

------------------------------
Testing YOLOv8n
------------------------------
Loading YOLO (yolov8n.pt)...
M

YOLOv8n is so much faster and lighter than the Faster R-CNN model. The biggest difference is the speed: YOLOv8n gets over 95 FPS on video, which is definitely fast enough for real-time, while the Faster R-CNN is stuck at a super slow 7.66 FPS. The single image inference test tells the same story, with YOLO being way quicker (71.65 ms) compared to the R-CNN model (352.62 ms)